# Build knowledge graph via deep learning

## Problem
What are shared entities in all defi whitepapers?

## Solution
With bag of words model, we will extract entities in whitepapers.

## Expected outcome
1. Top ten entities for each whitepaper are extracted.
2. Top ten trigram for each whitepaper are extracted.

Authors:
* Xiaoyuan Liu
* Neel Kovelamudi
* Zijian Xie
* Mu He
* Cuiqianhe Du
* Nicholas Lin
* Austin Wei

Principal Investigator: 
* Dawn Song

Date: Fall 2021

References: 
[Bag_of_words](https://www.analyticsvidhya.com/blog/2021/08/a-friendly-guide-to-nlp-bag-of-words-with-python-example/)

### Import library

In [1]:
import warnings
warnings.filterwarnings("ignore")
import pandas as pd
import numpy as np
import collections
import re
import os
import string
pd.set_option('display.max_colwidth', 200)
from sklearn.feature_extraction.text import CountVectorizer

In [2]:
#!pip install BeautifulSoup4
# import nltk
# nltk.download()  # Download text data sets, including stop words

In [3]:
# Import BeautifulSoup into your workspace
from bs4 import BeautifulSoup         

In [4]:
from nltk.corpus import stopwords # Import the stop word list
print(stopwords.words("english"))

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', '

### Whitepaper datasource

In [5]:
def read_whitepapers(filename):
    directory = "../whitepapers/top20_whitepapers/"
    texts = []
    for entry in os.scandir(directory):
        if (entry.path.endswith(filename) and entry.is_file()):
            a_dataframe = pd.read_csv(entry.path, names=[filename], sep="\n")
    a_dataframe.replace('', np.nan, inplace=True)
    a_dataframe.dropna(inplace=True)
    return a_dataframe

In [6]:
# bitcoin_filename="Bitcoin.txt"
# whitepapers = read_whitepapers(bitcoin_filename)
# # whitepapers.rename(columns={bitcoin_filename: "whitepapers"}, inplace=True)
# whitepapers

In [7]:
filenames = ['Algorand.txt', 'Avalanche.txt', 'Binance.txt', 'Bitcoin.txt', 'Cardano.txt', 'Chainlink.txt',
            'Crypto_com.txt', 'Ethereum.txt', 'FTX_token.txt', 'PolkaDot.txt', 'Polygon.txt', 'Ripple.txt', 
            'Solana.txt', 'Terra.txt', 'Tether.txt', 'Tron.txt', 'Uniswap.txt', 'Wrapped.txt']

In [8]:
def create_dataframe():
#     for i in filenames:
#         whitepaper = read_whitepapers(i)
#         whitepaper.rename(columns={i : "whitepapers"}, inplace=True)
#         whitepapers = whitepapers.append(whitepaper)
    df_from_each_whitepaper = (read_whitepapers(i) for i in filenames)
    whitepapers = pd.concat(df_from_each_whitepaper, ignore_index=True, axis=1)
    return whitepapers

### Create dataframe
Append whitepaper dataframe one after another

In [9]:
df_whitepapers = create_dataframe()
df_whitepapers.columns = filenames
df_whitepapers

,Algorand.txt,Avalanche.txt,Binance.txt,Bitcoin.txt,Cardano.txt,Chainlink.txt,Crypto_com.txt,Ethereum.txt,FTX_token.txt,PolkaDot.txt,Polygon.txt,Ripple.txt,Solana.txt,Terra.txt,Tether.txt,Tron.txt,Uniswap.txt,Wrapped.txt
0,ALGORAND AGREEMENT,Avalanche Platform,Binance Exchange,Bitcoin: A Peer-to-Peer Electronic Cash System,Ouroboros: A Provably Secure Proof-of-Stake Blockchain Protocol,Chainlink 2.0: Next Steps in the Evolution of,Crypto.com Whitepaper 1.03,HOME / WHITEPAPER,FTT Whitepaper,POLKADOT: VISION FOR A HETEROGENEOUS MULTI-CHAIN FRAMEWORK,Search or jump to… Pulls Issues Marketplace Explore,"RippleLabsInc,2014",Solana: A new architecture for a high,Terra Money:,,Advanced Decentralized Blockchain Platform,Uniswap v2 Core,
1,Super Fast and Partition Resilient Byzantine Agreement,2020/06/30,www.binance.com,Satoshi Nakamoto,Aggelos Kiayias∗ Alexander Russell† Bernardo David‡ Roman Oliynykov§,Decentralized Oracle Networks,August 2020,"Page last updated: January 30, 2022","Initially Released June 25, 2019",DRAFT 1,maticnetwork / whitepaper,The Ripple Protocol Consensus Algorithm,performance blockchain v0.8.14,Stability and Adoption,,Whitepaper Version: 2.0,Hayden Adams Noah Zinsmeister Dan Robinson,Wrapped Tokens
2,Jing Chen Sergey Gorbunov Silvio Micali Georgios Vlachos,"Kevin Sekniqi, Daniel Laine, Stephen Buttolph, and Emin Gu¨n Sirer",Whitepaper,satoshin@gmx.com,"July 20, 2019",Lorenz Breidenbach1 Christian Cachin2 Benedict Chan1,Version 1.03.17 - April 2021,On this page,1,DR.GAVINWOOD,Public Watch 12 Fork 29 Star 125,David Schwartz,Anatoly Yakovenko,"Evan Kereiakes, Do Kwon, Marco Di Maggio, Nicholas Platias",,TRON Protocol Version: 3.2,hayden@uniswap.org noah@uniswap.org dan@paradigm.xyz,A multi-institutional framework for tokenizing any asset
3,"{jing, sergey, silvio, georgios@algorand.com}","Abstract. ThispaperprovidesanarchitecturaloverviewoftheﬁrstreleaseoftheAvalancheplatform,",V1.2,www.bitcoin.org,Abstract,Alex Coventry1 Steve Ellis1 Ari Juels3 Farinaz Koushanfar4,This whitepaper is a working document that is subject to review and changes,Ethereum Whitepaper,Contents,"FOUNDER,ETHEREUM&PARITY",Code Issues 3 Pull requests 1 Actions Projects Wiki Security,This paper does not reflect the current state of the ledger consensus protocol or its,anatoly@solana.io,April 2019,,TRON DAO,March 2020,
4,"April 25, 2018","5 codenamed Avalanche Borealis. For details on the economics of the native token, labeled $AVAX, we",Intro 3,Abstract. A purely peer-to-peer version of electronic cash would allow online,"We present “Ouroboros,” the ﬁrst blockchain protocol based on proof of stake with rig-",Andrew Miller5 Brendan Magauran1 Daniel Moroz6,Crypto.com 2,"This introductory paper was originally published in 2013 by Vitalik Buterin, the founder of Ethereum,",1 Our Mission 4,GAVIN@PARITY.IO,master whitepaper / README.md Go to file,"david@ripple.com analysis. We will continue hosting this draft for historical interest, but it SHOULD NOT be","Legal Disclaimer NothinginthisWhitePaperisanoffertosell,orthesolicitationofanoffer",Abstract,,"December 10th, 2018, San Francisco",Abstract,Whitepaper v0.2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5327,NaN,NaN,NaN,NaN,NaN,tion.,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5328,NaN,NaN,NaN,NaN,NaN,• Customization: Diﬀerent users may have diﬀerent preferences in terms of trade-,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5329,NaN,NaN,NaN,NaN,NaN,"oﬀs among reliability, performance, and cost tradeoﬀs, and should be able to",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5330,NaN,NaN,NaN,NaN,NaN,express these preferences in terms of their selection of providers.,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### Data cleaning/preprocessing

In [10]:
def sentence_to_words(sentence):
    # Function to convert a raw sentence to a string of words
    # The input is a single string (a whitepaper sentence), and 
    # the output is a single string (a preprocessed sentence)
    
    # if the line is nah, return
    if pd.isnull(sentence):
        return sentence
    # 1. Remove HTML
    sentence = BeautifulSoup(sentence).get_text() 
    #
    # 2. Remove non-letters        
    letters_only = re.sub("[^a-zA-Z]", " ", sentence) 
    #
    # 3. Convert to lower case, split into individual words
    words = letters_only.lower().split()                             
    #
    # 4. In Python, searching a set is much faster than searching
    #   a list, so convert the stop words to a set
    stops = set(stopwords.words("english"))                  
    # 
    # 5. Remove stop words
    meaningful_words = [w for w in words if not w in stops]   
    #
    # 6. Join the words back into one string separated by space, 
    # and return the result.
    return( " ".join( meaningful_words ))

In [11]:
for i in filenames:
    df_whitepapers[i] = df_whitepapers[i].apply(lambda x: sentence_to_words(x))
df_whitepapers

,Algorand.txt,Avalanche.txt,Binance.txt,Bitcoin.txt,Cardano.txt,Chainlink.txt,Crypto_com.txt,Ethereum.txt,FTX_token.txt,PolkaDot.txt,Polygon.txt,Ripple.txt,Solana.txt,Terra.txt,Tether.txt,Tron.txt,Uniswap.txt,Wrapped.txt
0,algorand agreement,avalanche platform,binance exchange,bitcoin peer peer electronic cash system,ouroboros provably secure proof stake blockchain protocol,chainlink next steps evolution,crypto com whitepaper,home whitepaper,ftt whitepaper,polkadot vision heterogeneous multi chain framework,search jump pulls issues marketplace explore,ripplelabsinc,solana new architecture high,terra money,,advanced decentralized blockchain platform,uniswap v core,
1,super fast partition resilient byzantine agreement,,www binance com,satoshi nakamoto,aggelos kiayias alexander russell bernardo david roman oliynykov,decentralized oracle networks,august,page last updated january,initially released june,draft,maticnetwork whitepaper,ripple protocol consensus algorithm,performance blockchain v,stability adoption,,whitepaper version,hayden adams noah zinsmeister dan robinson,wrapped tokens
2,jing chen sergey gorbunov silvio micali georgios vlachos,kevin sekniqi daniel laine stephen buttolph emin gu n sirer,whitepaper,satoshin gmx com,july,lorenz breidenbach christian cachin benedict chan,version april,page,,dr gavinwood,public watch fork star,david schwartz,anatoly yakovenko,evan kereiakes kwon marco di maggio nicholas platias,,tron protocol version,hayden uniswap org noah uniswap org dan paradigm xyz,multi institutional framework tokenizing asset
3,jing sergey silvio georgios algorand com,abstract thispaperprovidesanarchitecturaloverviewofthe rstreleaseoftheavalancheplatform,v,www bitcoin org,abstract,alex coventry steve ellis ari juels farinaz koushanfar,whitepaper working document subject review changes,ethereum whitepaper,contents,founder ethereum parity,code issues pull requests actions projects wiki security,paper reflect current state ledger consensus protocol,anatoly solana io,april,,tron dao,march,
4,april,codenamed avalanche borealis details economics native token labeled avax,intro,abstract purely peer peer version electronic cash would allow online,present ouroboros rst blockchain protocol based proof stake rig,andrew miller brendan magauran daniel moroz,crypto com,introductory paper originally published vitalik buterin founder ethereum,mission,gavin parity io,master whitepaper readme md go file,david ripple com analysis continue hosting draft historical interest,legal disclaimer nothinginthiswhitepaperisanoffertosell orthesolicitationofanoffer,abstract,,december th san francisco,abstract,whitepaper v
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5327,NaN,NaN,NaN,NaN,NaN,tion,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5328,NaN,NaN,NaN,NaN,NaN,customization di erent users may di erent preferences terms trade,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5329,NaN,NaN,NaN,NaN,NaN,among reliability performance cost tradeo able,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5330,NaN,NaN,NaN,NaN,NaN,express preferences terms selection providers,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [12]:
def fit(vectorizer, df_column):
    # Function to fit a vectorizer to whitepaper
    # The input is vectorizer, and a df_column (whitepaper)
    # the output is a list of top 10 occurence
    # fit a column
    df_column.dropna(inplace=True)
    features = vectorizer.fit_transform(df_column)
    vocabulary = vectorizer.get_feature_names()
    features = features.toarray()
    
    # Sum up the counts of each vocabulary word
    count_sum = np.sum(features, axis=0)

    # For each, print the vocabulary word and the number of times it 
    # appears in the training set
    paired =  list(zip(vocabulary, count_sum))
    
    # reverse sort occurence of words
    feat_occ = sorted(paired, key = lambda x: x[1], reverse=True)
    return pd.DataFrame(list(feat_occ))

### CountVectorizer

In [13]:
vectorizer1 = CountVectorizer(stop_words='english')

### Count of each word in the vocabulary

In [14]:
bow_occ_from_each_whitepaper = (fit(vectorizer1, df_whitepapers[i]) for i in filenames)
df_bow_occ = pd.concat(bow_occ_from_each_whitepaper, ignore_index=True, axis=1)
df_bow_columns = [val for val in filenames for _ in (0, 1)]
df_bow_occ.columns = [i if not c % 2 else "count" for c, i in enumerate(df_bow_columns)]
df_bow_occ[:10]

,Algorand.txt,count,Avalanche.txt,count,Binance.txt,count,Bitcoin.txt,count,Cardano.txt,count,...,Terra.txt,count,Tether.txt,count,Tron.txt,count,Uniswap.txt,count,Wrapped.txt,count
0,period,103.0,avalanche,57.0,binance,32.0,block,51.0,cid,264.0,...,terra,79.0,tether,66.0,tron,105.0,uniswap,47.0,tokens,68.0
1,cid,79.0,network,31.0,exchange,29.0,hash,43.0,protocol,227.0,...,mining,50.0,tethers,57.0,trx,82.0,contract,45.0,wrapped,48.0
2,honest,67.0,platform,31.0,systems,23.0,transaction,37.0,honest,194.0,...,luna,48.0,fiat,56.0,network,72.0,price,35.0,wbtc,44.0
3,votes,65.0,protocols,28.0,bnb,22.0,transactions,32.0,sl,148.0,...,price,40.0,bitcoin,52.0,block,69.0,liquidity,33.0,merchant,43.0
4,value,62.0,consensus,27.0,trading,20.0,nodes,31.0,stake,143.0,...,rewards,37.0,users,42.0,account,65.0,eth,32.0,user,30.0
5,users,61.0,nodes,25.0,team,19.0,proof,27.0,chain,136.0,...,funding,27.0,exchange,36.0,bandwidth,43.0,pair,28.0,chain,28.0
6,step,56.0,state,25.0,founder,18.0,work,27.0,slot,129.0,...,unit,26.0,currency,32.0,token,43.0,pool,20.0,asset,27.0
7,protocol,37.0,set,24.0,bijietech,17.0,chain,25.0,fork,123.0,...,currency,24.0,exchanges,28.0,transaction,43.0,asset,19.0,custodian,26.0
8,time,36.0,avax,23.0,cz,17.0,attacker,22.0,slots,104.0,...,stability,24.0,limited,25.0,contract,41.0,tokens,17.0,ethereum,26.0
9,user,36.0,node,21.0,exchanges,15.0,network,21.0,length,102.0,...,stable,24.0,blockchain,23.0,smart,41.0,assets,16.0,token,25.0


### Count of shared entities among whitepapers

In [15]:
# First create a dictionary with key: whitepaper, value: list of words in df
dict_bow = df_bow_occ[filenames].dropna().to_dict('list')

In [23]:
# Then reverse the dictionary to have key: words, value: list of whitepaper
dict_bow_rev = {}
for k,v in dict_bow.items():
    for x in v:
        dict_bow_rev.setdefault(x,[]).append(k)

# Sort the dictionary with len of whitepapers' list
dict_bow_rev = dict(sorted(dict_bow_rev.items(), key=lambda item: len(item[1]), reverse=True)) 
# dict_bow_rev

In [25]:
# Count occurence of each word
dict_bow_count = {}

for key, values in dict_bow_rev.items():
    dict_bow_count[key] = len(values)

# Sort by occurence
dict_bow_count = dict(sorted(dict_bow_count.items(), key=lambda item: item[1], reverse=True))
# dict_bow_count

### Actual result
The previous column shows top 10 occurence in bag of words model among 20 whitepapers

### Trigram

In [18]:
vectorizer2 = CountVectorizer(stop_words='english',ngram_range=(3, 3))

### Count of each word in the vocabulary

In [19]:
tri_occ_from_each_whitepaper = (fit(vectorizer2, df_whitepapers[i]) for i in filenames)
df_tri_occ = pd.concat(tri_occ_from_each_whitepaper, ignore_index=True, axis=1)
df_tri_columns = [val for val in filenames for _ in (0, 1)]
df_tri_occ.columns = [i if not c % 2 else "count" for c, i in enumerate(df_tri_columns)]
df_tri_occ[:10]

,Algorand.txt,count,Avalanche.txt,count,Binance.txt,count,Bitcoin.txt,count,Cardano.txt,count,...,Terra.txt,count,Tether.txt,count,Tron.txt,count,Uniswap.txt,count,Wrapped.txt,count
0,cid cid cid,17.0,buttolph emin gu,8.0,july th july,4.0,hash hash hash,7.0,cid cid cid,23.0,...,unit mining rewards,17.0,audit flaws exchanges,4.0,tron virtual machine,11.0,liquidity pool share,6.0,asset backed tokens,6.0
1,certi ed value,10.0,daniel laine stephen,8.0,th july th,4.0,prev hash nonce,7.0,sl sl sl,15.0,...,fees luna burn,4.0,custodian reserve assets,4.0,virtual machine tvm,8.0,asset terms asset,3.0,atomic swap contract,3.0
2,votes value cid,9.0,emin gu sirer,8.0,allan yan product,2.0,tx tx tx,5.0,cid exp cid,9.0,...,rate luna burn,4.0,existing fiat pegging,4.0,create new account,7.0,basis point fee,3.0,erc token ethereum,3.0
3,soft votes value,8.0,kevin sekniqi daniel,8.0,binance coin bnb,2.0,block header block,4.0,cid lexp cid,9.0,...,changes unit mining,3.0,fiat currency held,4.0,dynamic network parameters,7.0,cid ti pi,3.0,new wrapped tokens,3.0
4,ed value period,7.0,laine stephen buttolph,8.0,bnb pay fees,2.0,hash nonce prev,4.0,leader selection process,9.0,...,long term commitment,3.0,fiat pegging systems,4.0,delegated proof stake,5.0,https eips ethereum,3.0,address secret hash,2.0
5,potentially certi ed,7.0,sekniqi daniel laine,8.0,bnb value burn,2.0,nonce prev hash,4.0,closed fork let,7.0,...,long term stable,3.0,flaws exchanges wallets,4.0,false notice set,5.0,org eips eip,3.0,aml kyc procedures,2.0
6,value cid period,6.0,stephen buttolph emin,8.0,bnb vesting plan,2.0,proof work chain,4.0,computer science pages,7.0,...,luna burn rate,3.0,limitations existing fiat,4.0,notice set false,5.0,price asset terms,3.0,asset case wbtc,2.0
7,honest users cert,5.0,forward looking statements,6.0,changpeng zhao ceo,2.0,majority cpu power,3.0,exp cid cid,7.0,...,target exchange rate,3.0,omni layer protocol,4.0,set false true,5.0,url https eips,3.0,atomic swap fee,2.0
8,honest users period,5.0,post quantum cryptography,3.0,cz years worked,2.0,owner owner owner,3.0,lecture notes computer,7.0,...,available mining power,2.0,currency held reserves,3.0,total vote reward,5.0,value liquidity pool,3.0,atomic swap use,2.0
9,sees soft votes,5.0,blockchain ned vm,2.0,english chinese japanese,2.0,proof work block,3.0,notes computer science,7.0,...,central banks governments,2.0,decentralized digital currency,3.0,true false notice,5.0,angeris et al,2.0,backed tokens usually,2.0


### Count of shared trigram among whitepapers

In [20]:
# First create a dictionary with key: whitepaper, value: list of words in df
dict_tri = df_tri_occ[filenames].dropna().to_dict('list')

In [21]:
# Then reverse the dictionary to have key: words, value: list of whitepaper
dict_tri_rev = {}
for k,v in dict_tri.items():
    for x in v:
        dict_tri_rev.setdefault(x,[]).append(k)

# Sort the dictionary with len of whitepapers' list
dict_tri_rev = dict(sorted(dict_tri_rev.items(), key=lambda item: len(item[1]), reverse=True)) 
dict_tri_rev

{'anti money laundering': ['Binance.txt',
  'Crypto_com.txt',
  'Tether.txt',
  'Wrapped.txt'],
 'https github com': ['Cardano.txt',
  'PolkaDot.txt',
  'Polygon.txt',
  'Tron.txt'],
 'cid cid cid': ['Algorand.txt', 'Cardano.txt', 'Chainlink.txt'],
 'publicly veri able': ['Algorand.txt', 'Cardano.txt', 'Chainlink.txt'],
 'bitcoin peer peer': ['Avalanche.txt', 'Bitcoin.txt', 'Ripple.txt'],
 'ethereum virtual machine': ['Avalanche.txt', 'Ethereum.txt', 'Tron.txt'],
 'en wikipedia org': ['Binance.txt', 'Tether.txt', 'Tron.txt'],
 'block header block': ['Bitcoin.txt', 'Ethereum.txt', 'Tron.txt'],
 'eth usd price': ['Chainlink.txt', 'Ethereum.txt', 'Uniswap.txt'],
 'wikipedia org wiki': ['Chainlink.txt', 'Tether.txt', 'Tron.txt'],
 'bitcoin org bitcoin': ['PolkaDot.txt', 'Tether.txt', 'Tron.txt'],
 'able random functions': ['Algorand.txt', 'Chainlink.txt'],
 'arxiv org abs': ['Algorand.txt', 'Ripple.txt'],
 'public secret key': ['Algorand.txt', 'Cardano.txt'],
 'veri able random': ['Algoran

In [22]:
# Count occurence of each word
dict_tri_count = {}

for key, values in dict_tri_rev.items():
    dict_tri_count[key] = len(values)

# Sort by occurence
dict_tri_count = dict(sorted(dict_tri_count.items(), key=lambda item: item[1], reverse=True))
dict_tri_count

{'anti money laundering': 4,
 'https github com': 4,
 'cid cid cid': 3,
 'publicly veri able': 3,
 'bitcoin peer peer': 3,
 'ethereum virtual machine': 3,
 'en wikipedia org': 3,
 'block header block': 3,
 'eth usd price': 3,
 'wikipedia org wiki': 3,
 'bitcoin org bitcoin': 3,
 'able random functions': 2,
 'arxiv org abs': 2,
 'public secret key': 2,
 'veri able random': 2,
 'cryptology eprint archive': 2,
 'eprint archive report': 2,
 'forward looking statements': 2,
 'computational resources required': 2,
 'deploy smart contracts': 2,
 'based trading volume': 2,
 'https en wikipedia': 2,
 'hash hash hash': 2,
 'proof work block': 2,
 'allowing willing parties': 2,
 'based cryptographic proof': 2,
 'block header transactions': 2,
 'computer communications security': 2,
 'computer science pages': 2,
 'cryptographic proof instead': 2,
 'directly need trusted': 2,
 'electronic payment based': 2,
 'ieee computer society': 2,
 'able secret sharing': 2,
 'long range attacks': 2,
 'advances

### Actual result
The previous column shows top 10 occurence in trigram model among 20 whitepapers